In [1]:
import os
import glob
import random
import numpy as np
import pandas as pd
from scipy.ndimage import gaussian_filter1d
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# ------------------------------------------------------------------------------
# 1. Strict Seeding
# ------------------------------------------------------------------------------
def set_strict_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"[Info] Seed fixed to {seed}")


# ------------------------------------------------------------------------------
# 2. Data Loading
# ------------------------------------------------------------------------------
def load_mhealth_dataset(data_dir, target_activities_map, column_names):
    full_dataset = {}
    file_list = sorted(glob.glob(os.path.join(data_dir, "mHealth_subject*.log")))

    if not file_list:
        print(f"[Warning] No mHealth logs found in {data_dir}")
        return {}

    print(f"Loading {len(file_list)} subjects from {data_dir}...")

    for file_path in file_list:
        file_name = os.path.basename(file_path)
        subj_part = file_name.split('.')[0]
        try:
            subj_id_num = int(''.join(filter(str.isdigit, subj_part)))
            subj_key = f"subject{subj_id_num}"
        except:
            subj_key = subj_part

        try:
            df = pd.read_csv(file_path, sep="\t", header=None)
            df = df.iloc[:, :len(column_names)]
            df.columns = column_names

            subj_data = {}
            for label_code, activity_name in target_activities_map.items():
                activity_df = df[df['activity_id'] == label_code].copy()
                if not activity_df.empty:
                    subj_data[activity_name] = activity_df.drop(columns=['activity_id'])

            full_dataset[subj_key] = subj_data
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            pass

    return full_dataset

def prepare_trial_list(label_config, full_data, target_map, feature_map):
    trial_list = []
    for subj, act_id, gt_count in label_config:
        act_name = target_map.get(act_id)
        feats = feature_map.get(act_id)

        if subj in full_data and act_name in full_data[subj]:
            # Get raw data (N, C)
            raw_df = full_data[subj][act_name][feats]
            raw_np = raw_df.values.astype(np.float32)

            # Normalize (Z-score per trial)
            mean = raw_np.mean(axis=0)
            std = raw_np.std(axis=0) + 1e-6
            norm_np = (raw_np - mean) / std

            trial_list.append({
                'data': norm_np,
                'count': float(gt_count),
                'meta': f"{subj}_{act_name}"
            })
        else:
            print(f"[Skip] Missing data for {subj} - {act_name}")

    return trial_list


# ------------------------------------------------------------------------------
# 3. Model Classes
# ------------------------------------------------------------------------------
class ManifoldEncoder(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=3):
        super().__init__()
        # Global pooling to capture window context if used primarily for reconstruction
        # But for sequence processing, we might want to keep temporal dimension
        # Here we use a architecture that preserves T somewhat or encodes window
        # For simplicity in this 'Trial-based' approach, let's keep it simple:
        # Encoder: (B, C, T) -> (B, D, T) mapping strictly time-wise

        self.net = nn.Sequential(
            nn.Conv1d(input_ch, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, latent_dim, 1) # Project to latent
        )

    def forward(self, x):
        # x: (B, C, T)
        z = self.net(x) # (B, D, T)
        z = z.transpose(1, 2) # (B, T, D) for easier handling later
        return z

class RateHead(nn.Module):
    def __init__(self, latent_dim=3, hidden=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )
    def forward(self, z):
        # z: (B, T, D)
        return self.net(z).squeeze(-1)  # (B, T)

class RateModel(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=64):
        super().__init__()
        self.encoder = ManifoldEncoder(input_ch, hidden_dim, latent_dim)
        self.rate_head = RateHead(latent_dim, hidden_dim)

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv1d) or isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
        # softplus로 양수 rate
        nn.init.constant_(self.rate_head.net[-1].bias, -2.0)

    def forward(self, x, mask=None):
        # x: (B, C, T)
        z = self.encoder(x)  # (B, T, D)

        rate_logits = self.rate_head(z)
        instant_rate = F.softplus(rate_logits)

        if mask is None:
            avg_rate = instant_rate.mean(dim=1)  # (B, )
        else:
            mask = mask.to(dtype=instant_rate.dtype, device=instant_rate.device)
            avg_rate = (instant_rate * mask).sum(dim=1) / (mask.sum(dim=1) + 1e-6)

        return avg_rate, z


# ------------------------------------------------------------------------------
# 4. Dataset & Collate
# ------------------------------------------------------------------------------
class TrialDataset(Dataset):
    """
    Wraps the loaded data to provide (sequence, count, meta)
    Compatible with Variable Length Collate
    """
    def __init__(self, trial_list):
        # trial_list: list of dicts {'data': np.array(T, C), 'count': float, 'meta': str}
        self.trials = trial_list

    def __len__(self):
        return len(self.trials)

    def __getitem__(self, idx):
        item = self.trials[idx]
        data = torch.tensor(item['data'], dtype=torch.float32).transpose(0, 1) # (C, T)
        count = torch.tensor(item['count'], dtype=torch.float32)
        return data, count, item['meta']

def collate_variable_length(batch):
    # batch: list of (data, count, meta)
    # data: (C, T)

    # 1. Find max length
    max_len = max([x[0].shape[1] for x in batch])
    C = batch[0][0].shape[0]

    padded_data, masks, counts, metas, lengths = [], [], [], [], []

    for data, count, meta in batch:
        T = data.shape[1]
        lengths.append(T)

        # Pad data
        pad_size = max_len - T
        if pad_size > 0:
            pad = torch.zeros(C, pad_size)
            d_padded = torch.cat([data, pad], dim=1)
            mask = torch.cat([torch.ones(T), torch.zeros(pad_size)], dim=0)
        else:
            d_padded = data
            mask = torch.ones(T)

        padded_data.append(d_padded)
        masks.append(mask)
        counts.append(count)
        metas.append(meta)

    return {
        "data": torch.stack(padded_data), # (B, C, T_max)
        "mask": torch.stack(masks),       # (B, T_max)
        "count": torch.stack(counts),     # (B,)
        "length": torch.tensor(lengths, dtype=torch.float32),  # (B,)
        "meta": metas
    }


# ------------------------------------------------------------------------------
# 5. Training
# ------------------------------------------------------------------------------
def train_one_epoch(model, loader, optimizer, config, device):
    model.train()
    stats = {k: 0.0 for k in ['loss', 'mae_rate', 'mae_count', 'pred_count', 'gt_count']}

    fs = config["fs"]

    for batch in loader:
        x = batch["data"].to(device)           # (B, C, T)
        mask = batch["mask"].to(device)        # (B, T)
        y_count = batch["count"].to(device)    # (B,)
        length = batch["length"].to(device)    # (B,)

        duration = length / fs                 # (B,)
        duration = torch.clamp(duration, min=1e-6)

        # GT rate
        y_rate = y_count / duration            # (B,)  reps/sec

        optimizer.zero_grad()

        rate_hat, z = model(x, mask)      # (B,)
        loss = F.mse_loss(rate_hat, y_rate)

        loss.backward()
        optimizer.step()

        # 평가용 count로 변환해서 모니터링
        count_hat = rate_hat * duration

        stats['loss'] += loss.item()
        stats['mae_rate'] += torch.abs(rate_hat - y_rate).mean().item()
        stats['mae_count'] += torch.abs(count_hat - y_count).mean().item()
        stats['pred_count'] += count_hat.mean().item()
        stats['gt_count'] += y_count.mean().item()

    n = len(loader)
    return {k: v/n for k, v in stats.items()}


# ------------------------------------------------------------------------------
# 6. Inference
# ------------------------------------------------------------------------------
def evaluation(model, trial_data, device, fs, gt_count=None):
    model.eval()
    with torch.no_grad():
        x_tensor = torch.tensor(trial_data, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
        T = trial_data.shape[0]
        duration = T / fs

        rate_hat, _ = model(x_tensor, mask=None)   # (1,)
        rate_val = rate_hat.item()
        count_val = rate_val * duration

    gt_str = f"{gt_count:.2f}" if gt_count is not None else "Unknown"
    print("-" * 50)
    print(f">>> Pred count: {count_val:.2f} | (rate={rate_val:.3f} reps/s, duration={duration:.2f}s) | GT: {gt_str}")
    print("-" * 50)
    return count_val


# ------------------------------------------------------------------------------
# 7. Main with CONFIG
# ------------------------------------------------------------------------------
def main():

    CONFIG = {
        "seed": 42,
        "data_dir": "/content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET",

        "COLUMN_NAMES": [
            'acc_chest_x', 'acc_chest_y', 'acc_chest_z',
            'ecg_1', 'ecg_2',
            'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
            'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
            'mag_ankle_x', 'mag_ankle_y', 'mag_ankle_z',
            'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
            'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z',
            'mag_arm_x', 'mag_arm_y', 'mag_arm_z',
            'activity_id'
        ],
        "TARGET_ACTIVITIES_MAP": {
            6: 'Waist bends forward',
            7: 'Frontal elevation of arms',
            8: 'Knees bending',
            12: 'Jump front & back'
        },
        "ACT_FEATURE_MAP": {
            6: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z', 'acc_arm_x', 'acc_arm_y', 'acc_arm_z'],
            7: ['acc_arm_x', 'acc_arm_y', 'acc_arm_z', 'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'],
            8: ['acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z', 'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z'],
            12: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z', 'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z']
        },

        # Training Params
        "epochs": 100,
        "lr": 5e-4,
        "batch_size": 64,  # Small batch size since sequences are long
        "fs": 50,

        # Loss Weights
        "lambda_count": 1.0,

        # Model Arch
        "hidden_dim": 128,
        "latent_dim": 64,

        "TRAIN_LABELS": [
            # (Subject_ID, Activity_ID, True_Count)
            ("subject1", 12, 40),
            ("subject2", 12, 45),
            ("subject3", 12, 42),
            ("subject4", 12, 42),
            ("subject5", 12, 40),
            ("subject6", 12, 42),
            ("subject7", 12, 38),
            ("subject8", 12, 41),
            ("subject9", 12, 41),
        ],

        "TEST_LABELS": [
            ("subject10", 12, 40),
        ]
    }

    # 1. Setup
    set_strict_seed(CONFIG["seed"])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    # 2. Load Raw Data
    full_data = load_mhealth_dataset(
        CONFIG["data_dir"],
        CONFIG["TARGET_ACTIVITIES_MAP"],
        CONFIG["COLUMN_NAMES"]
    )

    if not full_data:
        print("No data loaded. Exiting.")
        return

    # 3. Prepare Dataset
    train_data = prepare_trial_list(
        CONFIG["TRAIN_LABELS"],
        full_data,
        CONFIG["TARGET_ACTIVITIES_MAP"],
        CONFIG["ACT_FEATURE_MAP"]
    )

    test_data = prepare_trial_list(
        CONFIG["TEST_LABELS"],
        full_data,
        CONFIG["TARGET_ACTIVITIES_MAP"],
        CONFIG["ACT_FEATURE_MAP"]
    )

    if len(train_data) == 0:
        print("No training data found. Check CONFIG.")
        return

    train_loader = DataLoader(TrialDataset(train_data),
                              batch_size=CONFIG["batch_size"],
                              shuffle=True,
                              collate_fn=collate_variable_length)

    # 4. Model Init
    # Input channels depends on the activity (e.g., Jump has 6 feats)
    input_ch = train_data[0]['data'].shape[1]
    model = RateModel(input_ch=input_ch,
                     hidden_dim=CONFIG["hidden_dim"],
                     latent_dim=CONFIG["latent_dim"]).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["lr"])
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5) # 30에폭마다 학습률 절반으로

    # 5. Training Loop
    print("\nStarting Training...")
    for epoch in range(CONFIG["epochs"]):

        res = train_one_epoch(model, train_loader, optimizer, CONFIG, device)
        scheduler.step()

        if (epoch+1) % 10 == 0:
            print(f"Ep {epoch+1} | Loss: {res['loss']:.3f} | "
                  f"MAE_count: {res['mae_count']:.2f} | Pred: {res['pred_count']:.1f} vs GT: {res['gt_count']:.1f}")

    # 6. Test & Visualize
    print("\n[Inference on Test Set]")
    target_viz_id = 12
    viz_feat_names = CONFIG["ACT_FEATURE_MAP"].get(target_viz_id, ["feat"]*input_ch)

    if len(test_data) > 0:
        for item in test_data:
            print(f"Subject: {item['meta']} | GT Count: {item['count']}")
            evaluation(model, item['data'], device, fs=CONFIG["fs"], gt_count=item['count'])
    else:
        print("No test data defined. Showing train sample.")
        item = train_data[0]
        evaluation(model, item['data'], device, fs=CONFIG["fs"], gt_count=item['count'])


# ------------------------------------------------------------------------------
# 8. Stress Test
# ------------------------------------------------------------------------------
    def run_comprehensive_stress_test(model, trial_data, device, fs=50):
        model.eval()
        data_np = trial_data['data']
        T, C = data_np.shape
        dur_orig = T / fs

        print("="*60)
        print(f"[Comprehensive Stress Test] Target: {trial_data['meta']}")
        print("="*60)

        # 1. Length Invariance Test (Zero Padding)
        pad_len = int(T * 0.5)
        zeros = np.zeros((pad_len, C), dtype=np.float32)
        padded_data = np.concatenate([data_np, zeros], axis=0)
        T_pad = padded_data.shape[0]
        dur_pad = T_pad / fs

        with torch.no_grad():
            x_orig = torch.tensor(data_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)
            rate_orig, _ = model(x_orig, mask=None)
            count_orig = rate_orig.item() * dur_orig

            x_pad = torch.tensor(padded_data, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)
            rate_pad, _ = model(x_pad, mask=None)
            count_pad = rate_pad.item() * dur_pad

        print(f"1. [Length Invariance] Add 50% Silence")
        print(f"   - Original | Len: {dur_orig:.2f}s | Rate: {rate_orig.item():.3f} | Count: {count_orig:.2f}")
        print(f"   - Padded   | Len: {dur_pad:.2f}s | Rate: {rate_pad.item():.3f} | Count: {count_pad:.2f}")

        diff_len = abs(count_orig - count_pad)
        pct_len = (diff_len / count_orig) * 100
        if pct_len < 5.0:
            print(f"   ✅ PASS: Count maintained (Diff: {pct_len:.2f}%)")
        else:
            print(f"   ❌ WARNING: Count changed (Diff: {pct_len:.2f}%)")
        print("-" * 60)

        # 2. Consistency Test (Front vs Back Split)
        mid_point = T // 2
        front_np = data_np[:mid_point, :]
        back_np = data_np[mid_point:, :]

        with torch.no_grad():
            x_front = torch.tensor(front_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)
            rate_front = model(x_front, mask=None)[0].item()

            x_back = torch.tensor(back_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)
            rate_back = model(x_back, mask=None)[0].item()

        print(f"2. [Consistency Test] Front vs Back Rate")
        print(f"   - Full  (0~100%) : {rate_orig.item():.3f} reps/s")
        print(f"   - Front (0~50%)  : {rate_front:.3f} reps/s")
        print(f"   - Back  (50~100%): {rate_back:.3f} reps/s")

        diff_consist = abs(rate_front - rate_back) / (rate_orig.item() + 1e-6) * 100
        if diff_consist < 15.0:
            print(f"   ✅ PASS: Rate is consistent (Diff: {diff_consist:.1f}%)")
        else:
            print(f"   ⚠️ WARNING: Large rate difference (Diff: {diff_consist:.1f}%)")
        print("="*60)

    run_comprehensive_stress_test(model, test_data[0], device, fs=CONFIG["fs"])

if __name__ == "__main__":
    main()

[Info] Seed fixed to 42
Device: cuda
Loading 10 subjects from /content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET...

Starting Training...
Ep 10 | Loss: 0.536 | MAE_count: 14.66 | Pred: 55.9 vs GT: 41.2
Ep 20 | Loss: 0.137 | MAE_count: 7.09 | Pred: 34.1 vs GT: 41.2
Ep 30 | Loss: 0.012 | MAE_count: 1.79 | Pred: 39.7 vs GT: 41.2
Ep 40 | Loss: 0.004 | MAE_count: 1.08 | Pred: 41.6 vs GT: 41.2
Ep 50 | Loss: 0.002 | MAE_count: 0.74 | Pred: 41.2 vs GT: 41.2
Ep 60 | Loss: 0.001 | MAE_count: 0.58 | Pred: 41.2 vs GT: 41.2
Ep 70 | Loss: 0.001 | MAE_count: 0.63 | Pred: 41.5 vs GT: 41.2
Ep 80 | Loss: 0.001 | MAE_count: 0.58 | Pred: 40.9 vs GT: 41.2
Ep 90 | Loss: 0.001 | MAE_count: 0.48 | Pred: 41.4 vs GT: 41.2
Ep 100 | Loss: 0.001 | MAE_count: 0.45 | Pred: 41.3 vs GT: 41.2

[Inference on Test Set]
Subject: subject10_Jump front & back | GT Count: 40.0
--------------------------------------------------
>>> Pred count: 39.52 | (rate=1.929 reps/s, duration=20.48s) | GT: 40.00
---------------